# Addition_rnn

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


## Data Representation

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
            
        #print(x)
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)
print(ctable)

## Data Generation

In [5]:
def Generate_Data(DIGITS = 3, DATA_SIZE = 50000):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b)))
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        expected.append(ans)
    print('Total addition questions:', len(questions))
    
    return questions, expected

## Feature Engineering

In [6]:
def Vectorization(questions, expected):
    print('Vectorization...')
    x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(expected):
        y[i] = ctable.encode(sentence, DIGITS + 1)

    return x, y

## Get Training Data and Validation Data
* total data = 50000, 45000 for training, 5000 for validation

In [7]:
questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## Build Model

In [8]:
def Build_Model(RNN, HIDDEN_SIZE, BATCH_SIZE, LAYERS, MAXLEN, DIGITS, chars):
    print('Build model...')
    model = Sequential()
    # "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
    # Note: In a situation where your input sequences have a variable length,
    # use input_shape=(None, num_feature).
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    # As the decoder RNN's input, repeatedly provide with the last output of
    # RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
    # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
    model.add(layers.RepeatVector(DIGITS + 1))
    # The decoder RNN could be multiple layers stacked or a single layer.
    for _ in range(LAYERS):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))

    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    
    return model

In [9]:
def Train_Step(model, epoch, REVERSE = True):
    for iteration in range(1, epoch):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(x_val, y_val))
        # Select 10 samples from the validation set at random so we can visualize
        # errors.
        for i in range(10):
            ind = np.random.randint(0, len(x_val))
            rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
            preds = model.predict_classes(rowx, verbose=0)
            q = ctable.decode(rowx[0])
            correct = ctable.decode(rowy[0])
            guess = ctable.decode(preds[0], calc_argmax=False)
            print('Q', q[::-1] if REVERSE else q, end=' ')
            print('T', correct, end=' ')
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print(guess)

## String Matching
* training 100 epochs

In [10]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [11]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 253us/step - loss: 1.8874 - acc: 0.3225 - val_loss: 1.7935 - val_acc: 0.3398
Q 465+403 T 868  ☒ 106 
Q 80+128  T 208  ☒ 100 
Q 27+874  T 901  ☒ 106 
Q 154+97  T 251  ☒ 102 
Q 622+671 T 1293 ☒ 106 
Q 69+400  T 469  ☒ 100 
Q 391+79  T 470  ☒ 100 
Q 274+104 T 378  ☒ 106 
Q 7+524   T 531  ☒ 721 
Q 525+30  T 555  ☒ 106 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 1.7306 - acc: 0.3617 - val_loss: 1.6586 - val_acc: 0.3801
Q 35+773  T 808  ☒ 777 
Q 951+9   T 960  ☒ 100 
Q 706+258 T 964  ☒ 100 
Q 60+58   T 118  ☒ 16  
Q 837+168 T 1005 ☒ 107 
Q 19+859  T 878  ☒ 502 
Q 480+760 T 1240 ☒ 1376
Q 91+512  T 603  ☒ 102 
Q 942+1   T 943  ☒ 102 
Q 23+256  T 279  ☒ 367 

------------------

45000/45000 [==============================] - 7s 160us/step - loss: 0.1773 - acc: 0.9667 - val_loss: 0.1570 - val_acc: 0.9712
Q 603+603 T 1206 ☒ 1216
Q 859+13  T 872  ☑ 872 
Q 88+786  T 874  ☑ 874 
Q 104+16  T 120  ☑ 120 
Q 95+534  T 629  ☑ 629 
Q 581+18  T 599  ☒ 699 
Q 178+22  T 200  ☑ 200 
Q 32+327  T 359  ☑ 359 
Q 444+848 T 1292 ☑ 1292
Q 231+79  T 310  ☑ 310 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 159us/step - loss: 0.1473 - acc: 0.9705 - val_loss: 0.1227 - val_acc: 0.9786
Q 77+985  T 1062 ☑ 1062
Q 351+26  T 377  ☑ 377 
Q 953+92  T 1045 ☑ 1045
Q 85+940  T 1025 ☑ 1025
Q 383+241 T 624  ☑ 624 
Q 15+486  T 501  ☑ 501 
Q 42+763  T 805  ☑ 805 
Q 729+900 T 1629 ☑ 1629
Q 1+578   T 579  ☑ 579 
Q 318+723 T 1041 ☒ 1031

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 161us/step - loss: 0.0470 - acc: 0.9868 - val_loss: 0.0159 - val_acc: 0.9976
Q 6+742   T 748  ☑ 748 
Q 531+42  T 573  ☑ 573 
Q 2+36    T 38   ☒ 48  
Q 790+951 T 1741 ☑ 1741
Q 122+412 T 534  ☑ 534 
Q 836+7   T 843  ☑ 843 
Q 59+449  T 508  ☑ 508 
Q 84+576  T 660  ☑ 660 
Q 704+105 T 809  ☑ 809 
Q 36+678  T 714  ☑ 714 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 162us/step - loss: 0.0119 - acc: 0.9989 - val_loss: 0.0118 - val_acc: 0.9984
Q 852+60  T 912  ☑ 912 
Q 280+80  T 360  ☑ 360 
Q 478+34  T 512  ☑ 512 
Q 388+4   T 392  ☑ 392 
Q 5+415   T 420  ☑ 420 
Q 16+731  T 747  ☑ 747 
Q 391+47  T 438  ☑ 438 
Q 833+48  T 881  ☑ 881 
Q 264+77  T 341  ☑ 341 
Q 928+66  T 994  ☑ 994 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 161us/step - loss: 0.0035 - acc: 0.9998 - val_loss: 0.0042 - val_acc: 0.9995
Q 874+961 T 1835 ☑ 1835
Q 13+332  T 345  ☑ 345 
Q 68+754  T 822  ☑ 822 
Q 433+985 T 1418 ☑ 1418
Q 810+533 T 1343 ☑ 1343
Q 50+552  T 602  ☑ 602 
Q 442+21  T 463  ☑ 463 
Q 709+64  T 773  ☑ 773 
Q 705+58  T 763  ☑ 763 
Q 626+6   T 632  ☑ 632 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 0.0040 - acc: 0.9995 - val_loss: 0.0173 - val_acc: 0.9953
Q 51+891  T 942  ☑ 942 
Q 406+5   T 411  ☑ 411 
Q 69+397  T 466  ☑ 466 
Q 706+60  T 766  ☑ 766 
Q 88+82   T 170  ☑ 170 
Q 76+304  T 380  ☑ 380 
Q 32+503  T 535  ☑ 535 
Q 44+136  T 180  ☑ 180 
Q 911+21  T 932  ☑ 932 
Q 20+614  T 634  ☑ 634 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 159us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 0.9997
Q 167+90  T 257  ☑ 257 
Q 253+6   T 259  ☑ 259 
Q 4+525   T 529  ☑ 529 
Q 48+103  T 151  ☑ 151 
Q 879+50  T 929  ☑ 929 
Q 53+94   T 147  ☑ 147 
Q 746+845 T 1591 ☑ 1591
Q 864+20  T 884  ☑ 884 
Q 777+300 T 1077 ☑ 1077
Q 188+94  T 282  ☑ 282 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 157us/step - loss: 0.0297 - acc: 0.9922 - val_loss: 0.1963 - val_acc: 0.9391
Q 628+408 T 1036 ☒ 1046
Q 71+911  T 982  ☑ 982 
Q 156+904 T 1060 ☑ 1060
Q 42+737  T 779  ☑ 779 
Q 866+64  T 930  ☑ 930 
Q 804+408 T 1212 ☒ 1312
Q 91+112  T 203  ☑ 203 
Q 77+95   T 172  ☑ 172 
Q 681+0   T 681  ☑ 681 
Q 521+81  T 602  ☑ 602 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 163us/step - loss: 0.0207 - acc: 0.9940 - val_loss: 0.1441 - val_acc: 0.9565
Q 60+925  T 985  ☑ 985 
Q 386+40  T 426  ☑ 426 
Q 47+923  T 970  ☒ 960 
Q 27+915  T 942  ☑ 942 
Q 94+29   T 123  ☑ 123 
Q 895+2   T 897  ☑ 897 
Q 292+589 T 881  ☒ 891 
Q 411+540 T 951  ☑ 951 
Q 732+51  T 783  ☑ 783 
Q 696+8   T 704  ☑ 704 

--------------------------------------------------
Iteration 72
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 0.0248 - acc: 0.9929 - val_loss: 0.0043 - val_acc: 0.9989
Q 745+338 T 1083 ☑ 1083
Q 370+61  T 431  ☑ 431 
Q 284+142 T 426  ☑ 426 
Q 193+772 T 965  ☑ 965 
Q 948+98  T 1046 ☑ 1046
Q 90+543  T 633  ☑ 633 
Q 965+35  T 1000 ☑ 1000
Q 264+38  T 302  ☑ 302 
Q 48+676  T 724  ☑ 724 
Q 391+0   T 391  ☑ 391 

--------------------------------------------------
Iteration 73
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 160us/step - loss: 8.8685e-04 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9996
Q 917+61  T 978  ☑ 978 
Q 552+73  T 625  ☑ 625 
Q 38+476  T 514  ☑ 514 
Q 919+96  T 1015 ☑ 1015
Q 338+729 T 1067 ☑ 1067
Q 890+936 T 1826 ☑ 1826
Q 34+367  T 401  ☑ 401 
Q 50+763  T 813  ☑ 813 
Q 107+636 T 743  ☑ 743 
Q 125+85  T 210  ☑ 210 

--------------------------------------------------
Iteration 86
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 159us/step - loss: 7.5066e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9996
Q 80+54   T 134  ☑ 134 
Q 408+655 T 1063 ☑ 1063
Q 931+61  T 992  ☑ 992 
Q 34+814  T 848  ☑ 848 
Q 663+73  T 736  ☑ 736 
Q 79+33   T 112  ☑ 112 
Q 472+596 T 1068 ☑ 1068
Q 50+256  T 306  ☑ 306 
Q 82+614  T 696  ☑ 696 
Q 17+620  T 637  ☑ 637 

--------------------------------------------------
Iteration 87
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/4500

45000/45000 [==============================] - 7s 159us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9998
Q 928+7   T 935  ☑ 935 
Q 680+6   T 686  ☑ 686 
Q 918+447 T 1365 ☑ 1365
Q 775+889 T 1664 ☑ 1664
Q 92+239  T 331  ☑ 331 
Q 11+972  T 983  ☑ 983 
Q 204+466 T 670  ☑ 670 
Q 323+253 T 576  ☑ 576 
Q 404+51  T 455  ☑ 455 
Q 3+977   T 980  ☑ 980 


## Testing

In [12]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [17]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 4+68    T 72   ☑ 72  
Q 1+55    T 56   ☑ 56  
Q 3+0     T 3    ☑ 3   
Q 497+2   T 499  ☑ 499 
Q 23+503  T 526  ☑ 526 
Q 692+506 T 1198 ☑ 1198
Q 67+6    T 73   ☑ 73  
Q 1+88    T 89   ☑ 89  
Q 52+405  T 457  ☑ 457 
Q 49+2    T 51   ☑ 51  
Testing Accuracy :  0.99


## Result
### * 利用LSTM做encoder-decoder的seq2seq架構來實現加法器
### * Training data總數為50000筆，45000為training，5000為validation
### * Batch size = 128
### * RNN hidden layer = 128
### * Optimizer使用adam
### * Epochs = 100
### * Validation result可達99.97%
### * Testing data總數為1000筆，Testing accuracy可達 99%

## Other Discussion
* 實驗使用不同的epoch和batch size訓練
* 實驗不同位數的數字
* 實驗"more number added"

### 1.1 實驗使用不同的epoch和batch size訓練
* with batch = 64, epoch = 100

In [18]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 64
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [19]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 349us/step - loss: 1.8389 - acc: 0.3343 - val_loss: 1.7258 - val_acc: 0.3617
Q 284+142 T 426  ☒ 100 
Q 35+689  T 724  ☒ 100 
Q 837+168 T 1005 ☒ 1371
Q 449+366 T 815  ☒ 106 
Q 606+524 T 1130 ☒ 106 
Q 125+85  T 210  ☒ 500 
Q 657+7   T 664  ☒ 770 
Q 7+415   T 422  ☒ 143 
Q 80+556  T 636  ☒ 500 
Q 10+98   T 108  ☒ 100 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 316us/step - loss: 1.6013 - acc: 0.4036 - val_loss: 1.4939 - val_acc: 0.4430
Q 98+807  T 905  ☒ 903 
Q 85+680  T 765  ☒ 745 
Q 38+745  T 783  ☒ 748 
Q 482+82  T 564  ☒ 811 
Q 155+318 T 473  ☒ 506 
Q 498+88  T 586  ☒ 546 
Q 429+1   T 430  ☒ 445 
Q 846+145 T 991  ☒ 1116
Q 65+796  T 861  ☒ 702 
Q 172+33  T 205  ☒ 247 

-----------------

45000/45000 [==============================] - 14s 312us/step - loss: 0.0465 - acc: 0.9907 - val_loss: 0.0369 - val_acc: 0.9932
Q 525+700 T 1225 ☑ 1225
Q 29+365  T 394  ☑ 394 
Q 91+460  T 551  ☑ 551 
Q 492+728 T 1220 ☑ 1220
Q 221+987 T 1208 ☑ 1208
Q 50+456  T 506  ☑ 506 
Q 210+825 T 1035 ☑ 1035
Q 886+66  T 952  ☑ 952 
Q 1+0     T 1    ☑ 1   
Q 60+193  T 253  ☑ 253 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 313us/step - loss: 0.0531 - acc: 0.9865 - val_loss: 0.0345 - val_acc: 0.9932
Q 21+919  T 940  ☑ 940 
Q 86+548  T 634  ☑ 634 
Q 34+820  T 854  ☑ 854 
Q 756+428 T 1184 ☑ 1184
Q 802+754 T 1556 ☑ 1556
Q 91+881  T 972  ☑ 972 
Q 4+601   T 605  ☑ 605 
Q 92+471  T 563  ☑ 563 
Q 59+83   T 142  ☑ 142 
Q 931+34  T 965  ☑ 965 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 14s 315us/step - loss: 0.0091 - acc: 0.9987 - val_loss: 0.0073 - val_acc: 0.9986
Q 461+31  T 492  ☑ 492 
Q 148+75  T 223  ☑ 223 
Q 8+624   T 632  ☑ 632 
Q 761+817 T 1578 ☑ 1578
Q 762+360 T 1122 ☑ 1122
Q 799+847 T 1646 ☑ 1646
Q 919+94  T 1013 ☑ 1013
Q 266+91  T 357  ☑ 357 
Q 63+595  T 658  ☑ 658 
Q 2+765   T 767  ☑ 767 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 313us/step - loss: 0.0047 - acc: 0.9995 - val_loss: 0.0127 - val_acc: 0.9966
Q 777+43  T 820  ☑ 820 
Q 76+662  T 738  ☑ 738 
Q 700+678 T 1378 ☑ 1378
Q 786+43  T 829  ☑ 829 
Q 439+0   T 439  ☑ 439 
Q 435+3   T 438  ☑ 438 
Q 59+544  T 603  ☑ 603 
Q 806+43  T 849  ☑ 849 
Q 43+3    T 46   ☑ 46  
Q 7+611   T 618  ☑ 618 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 14s 318us/step - loss: 0.0270 - acc: 0.9916 - val_loss: 0.0168 - val_acc: 0.9952
Q 689+852 T 1541 ☑ 1541
Q 929+46  T 975  ☑ 975 
Q 536+71  T 607  ☑ 607 
Q 452+12  T 464  ☑ 464 
Q 388+47  T 435  ☑ 435 
Q 607+91  T 698  ☑ 698 
Q 484+829 T 1313 ☑ 1313
Q 504+646 T 1150 ☑ 1150
Q 77+95   T 172  ☑ 172 
Q 111+922 T 1033 ☑ 1033

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 312us/step - loss: 0.0033 - acc: 0.9995 - val_loss: 0.0033 - val_acc: 0.9994
Q 69+903  T 972  ☑ 972 
Q 30+90   T 120  ☑ 120 
Q 93+470  T 563  ☑ 563 
Q 295+437 T 732  ☑ 732 
Q 444+81  T 525  ☑ 525 
Q 245+571 T 816  ☑ 816 
Q 999+960 T 1959 ☑ 1959
Q 17+747  T 764  ☑ 764 
Q 83+376  T 459  ☑ 459 
Q 3+977   T 980  ☑ 980 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 14s 312us/step - loss: 0.0287 - acc: 0.9919 - val_loss: 0.0049 - val_acc: 0.9990
Q 31+440  T 471  ☑ 471 
Q 647+889 T 1536 ☑ 1536
Q 84+52   T 136  ☑ 136 
Q 528+6   T 534  ☑ 534 
Q 50+952  T 1002 ☑ 1002
Q 417+94  T 511  ☑ 511 
Q 735+25  T 760  ☑ 760 
Q 46+171  T 217  ☑ 217 
Q 69+1    T 70   ☑ 70  
Q 98+94   T 192  ☑ 192 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 318us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0026 - val_acc: 0.9997
Q 711+88  T 799  ☑ 799 
Q 39+46   T 85   ☑ 85  
Q 710+7   T 717  ☑ 717 
Q 6+533   T 539  ☑ 539 
Q 3+634   T 637  ☑ 637 
Q 116+93  T 209  ☑ 209 
Q 729+900 T 1629 ☑ 1629
Q 135+23  T 158  ☑ 158 
Q 304+35  T 339  ☑ 339 
Q 37+19   T 56   ☑ 56  

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 14s 309us/step - loss: 9.2229e-04 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 0.9996
Q 640+474 T 1114 ☑ 1114
Q 728+951 T 1679 ☑ 1679
Q 458+0   T 458  ☑ 458 
Q 94+945  T 1039 ☑ 1039
Q 237+97  T 334  ☑ 334 
Q 20+468  T 488  ☑ 488 
Q 884+854 T 1738 ☑ 1738
Q 42+265  T 307  ☑ 307 
Q 370+28  T 398  ☑ 398 
Q 721+834 T 1555 ☑ 1555

--------------------------------------------------
Iteration 72
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 309us/step - loss: 8.5425e-04 - acc: 0.9999 - val_loss: 0.0017 - val_acc: 0.9997
Q 984+912 T 1896 ☑ 1896
Q 5+275   T 280  ☑ 280 
Q 1+504   T 505  ☑ 505 
Q 526+519 T 1045 ☑ 1045
Q 29+716  T 745  ☑ 745 
Q 196+0   T 196  ☑ 196 
Q 257+330 T 587  ☑ 587 
Q 19+418  T 437  ☑ 437 
Q 685+43  T 728  ☑ 728 
Q 13+17   T 30   ☑ 30  

--------------------------------------------------
Iteration 73
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45

45000/45000 [==============================] - 15s 325us/step - loss: 0.0062 - acc: 0.9986 - val_loss: 0.0027 - val_acc: 0.9996
Q 99+514  T 613  ☑ 613 
Q 456+51  T 507  ☑ 507 
Q 411+225 T 636  ☑ 636 
Q 8+622   T 630  ☑ 630 
Q 0+434   T 434  ☑ 434 
Q 803+44  T 847  ☑ 847 
Q 366+45  T 411  ☑ 411 
Q 89+483  T 572  ☑ 572 
Q 247+26  T 273  ☑ 273 
Q 898+45  T 943  ☑ 943 

--------------------------------------------------
Iteration 86
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 321us/step - loss: 9.2487e-04 - acc: 0.9999 - val_loss: 0.0017 - val_acc: 0.9998
Q 32+273  T 305  ☑ 305 
Q 2+175   T 177  ☑ 177 
Q 624+369 T 993  ☑ 993 
Q 474+40  T 514  ☑ 514 
Q 659+638 T 1297 ☑ 1297
Q 888+9   T 897  ☑ 897 
Q 806+4   T 810  ☑ 810 
Q 63+595  T 658  ☑ 658 
Q 63+516  T 579  ☑ 579 
Q 471+27  T 498  ☑ 498 

--------------------------------------------------
Iteration 87
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 

45000/45000 [==============================] - 14s 316us/step - loss: 0.0051 - acc: 0.9987 - val_loss: 0.0021 - val_acc: 0.9996
Q 364+831 T 1195 ☑ 1195
Q 970+284 T 1254 ☑ 1254
Q 73+69   T 142  ☑ 142 
Q 52+398  T 450  ☑ 450 
Q 23+4    T 27   ☑ 27  
Q 736+130 T 866  ☑ 866 
Q 572+67  T 639  ☑ 639 
Q 812+70  T 882  ☑ 882 
Q 783+46  T 829  ☑ 829 
Q 204+72  T 276  ☑ 276 


In [20]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 4+68    T 72   ☑ 72  
Q 1+55    T 56   ☑ 56  
Q 3+0     T 3    ☑ 3   
Q 497+2   T 499  ☑ 499 
Q 23+503  T 526  ☑ 526 
Q 692+506 T 1198 ☑ 1198
Q 67+6    T 73   ☑ 73  
Q 1+88    T 89   ☑ 89  
Q 52+405  T 457  ☑ 457 
Q 49+2    T 51   ☑ 51  
Testing Accuracy :  0.991


### 1.2 實驗使用不同的epoch和batch size訓練
* with batch = 256, epoch = 100

In [29]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 256
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [30]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 6s 123us/step - loss: 1.9576 - acc: 0.3042 - val_loss: 1.8133 - val_acc: 0.3380
Q 993+8   T 1001 ☒ 105 
Q 696+36  T 732  ☒ 105 
Q 920+734 T 1654 ☒ 1101
Q 910+42  T 952  ☒ 105 
Q 933+20  T 953  ☒ 105 
Q 322+4   T 326  ☒ 12  
Q 119+845 T 964  ☒ 111 
Q 429+46  T 475  ☒ 105 
Q 421+787 T 1208 ☒ 110 
Q 122+673 T 795  ☒ 105 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 83us/step - loss: 1.7824 - acc: 0.3451 - val_loss: 1.7482 - val_acc: 0.3515
Q 67+475  T 542  ☒ 107 
Q 323+253 T 576  ☒ 133 
Q 713+91  T 804  ☒ 102 
Q 680+6   T 686  ☒ 177 
Q 4+170   T 174  ☒ 12  
Q 338+70  T 408  ☒ 107 
Q 253+956 T 1209 ☒ 102 
Q 552+82  T 634  ☒ 105 
Q 888+9   T 897  ☒ 100 
Q 231+353 T 584  ☒ 132 

--------------------

45000/45000 [==============================] - 4s 82us/step - loss: 0.8525 - acc: 0.6973 - val_loss: 0.8497 - val_acc: 0.6902
Q 99+606  T 705  ☒ 798 
Q 721+64  T 785  ☒ 787 
Q 709+54  T 763  ☒ 762 
Q 471+704 T 1175 ☒ 1190
Q 558+435 T 993  ☒ 909 
Q 475+9   T 484  ☑ 484 
Q 90+539  T 629  ☒ 623 
Q 64+548  T 612  ☒ 613 
Q 972+7   T 979  ☒ 989 
Q 665+9   T 674  ☑ 674 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 82us/step - loss: 0.7889 - acc: 0.7205 - val_loss: 0.7679 - val_acc: 0.7263
Q 550+573 T 1123 ☒ 1125
Q 6+746   T 752  ☒ 753 
Q 745+929 T 1674 ☒ 1675
Q 29+75   T 104  ☒ 10  
Q 216+72  T 288  ☑ 288 
Q 666+392 T 1058 ☒ 1055
Q 907+1   T 908  ☒ 907 
Q 45+458  T 503  ☒ 502 
Q 463+0   T 463  ☒ 464 
Q 60+42   T 102  ☒ 103 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 4s 84us/step - loss: 0.0961 - acc: 0.9866 - val_loss: 0.0955 - val_acc: 0.9845
Q 922+4   T 926  ☑ 926 
Q 99+921  T 1020 ☒ 1010
Q 67+660  T 727  ☑ 727 
Q 28+607  T 635  ☑ 635 
Q 52+36   T 88   ☑ 88  
Q 393+89  T 482  ☑ 482 
Q 171+659 T 830  ☑ 830 
Q 79+33   T 112  ☑ 112 
Q 58+77   T 135  ☑ 135 
Q 756+707 T 1463 ☑ 1463

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 83us/step - loss: 0.0832 - acc: 0.9892 - val_loss: 0.0842 - val_acc: 0.9867
Q 49+383  T 432  ☑ 432 
Q 634+0   T 634  ☑ 634 
Q 514+534 T 1048 ☑ 1048
Q 84+828  T 912  ☑ 912 
Q 0+408   T 408  ☑ 408 
Q 476+24  T 500  ☑ 500 
Q 440+75  T 515  ☑ 515 
Q 374+281 T 655  ☑ 655 
Q 6+462   T 468  ☑ 468 
Q 888+422 T 1310 ☑ 1310

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 4s 81us/step - loss: 0.0185 - acc: 0.9989 - val_loss: 0.0211 - val_acc: 0.9977
Q 801+4   T 805  ☑ 805 
Q 430+16  T 446  ☑ 446 
Q 430+197 T 627  ☑ 627 
Q 70+502  T 572  ☑ 572 
Q 11+658  T 669  ☑ 669 
Q 73+726  T 799  ☑ 799 
Q 999+788 T 1787 ☑ 1787
Q 993+321 T 1314 ☑ 1314
Q 318+745 T 1063 ☑ 1063
Q 723+848 T 1571 ☑ 1571

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 80us/step - loss: 0.0170 - acc: 0.9990 - val_loss: 0.0206 - val_acc: 0.9974
Q 47+676  T 723  ☑ 723 
Q 571+529 T 1100 ☑ 1100
Q 41+56   T 97   ☑ 97  
Q 759+41  T 800  ☑ 800 
Q 567+915 T 1482 ☑ 1482
Q 62+155  T 217  ☑ 217 
Q 661+58  T 719  ☑ 719 
Q 181+5   T 186  ☑ 186 
Q 485+39  T 524  ☑ 524 
Q 861+9   T 870  ☑ 870 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 4s 82us/step - loss: 0.0068 - acc: 0.9997 - val_loss: 0.0094 - val_acc: 0.9988
Q 82+615  T 697  ☑ 697 
Q 878+65  T 943  ☑ 943 
Q 277+90  T 367  ☑ 367 
Q 1+247   T 248  ☑ 248 
Q 8+231   T 239  ☑ 239 
Q 594+26  T 620  ☑ 620 
Q 21+919  T 940  ☑ 940 
Q 801+6   T 807  ☑ 807 
Q 993+321 T 1314 ☑ 1314
Q 91+510  T 601  ☑ 601 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 82us/step - loss: 0.0064 - acc: 0.9997 - val_loss: 0.0093 - val_acc: 0.9990
Q 83+233  T 316  ☑ 316 
Q 68+10   T 78   ☑ 78  
Q 27+553  T 580  ☑ 580 
Q 474+611 T 1085 ☑ 1085
Q 999+932 T 1931 ☑ 1931
Q 459+8   T 467  ☑ 467 
Q 264+96  T 360  ☑ 360 
Q 88+44   T 132  ☑ 132 
Q 124+62  T 186  ☑ 186 
Q 75+34   T 109  ☑ 109 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 4s 83us/step - loss: 0.0034 - acc: 0.9999 - val_loss: 0.0058 - val_acc: 0.9991
Q 85+602  T 687  ☑ 687 
Q 19+353  T 372  ☑ 372 
Q 25+511  T 536  ☑ 536 
Q 9+735   T 744  ☑ 744 
Q 724+505 T 1229 ☑ 1229
Q 152+125 T 277  ☑ 277 
Q 99+213  T 312  ☑ 312 
Q 678+3   T 681  ☑ 681 
Q 756+8   T 764  ☑ 764 
Q 589+298 T 887  ☑ 887 

--------------------------------------------------
Iteration 72
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 81us/step - loss: 0.0033 - acc: 0.9999 - val_loss: 0.0059 - val_acc: 0.9990
Q 336+37  T 373  ☑ 373 
Q 29+166  T 195  ☑ 195 
Q 874+99  T 973  ☑ 973 
Q 65+471  T 536  ☑ 536 
Q 14+827  T 841  ☑ 841 
Q 20+421  T 441  ☑ 441 
Q 37+417  T 454  ☑ 454 
Q 36+562  T 598  ☑ 598 
Q 524+584 T 1108 ☑ 1108
Q 13+575  T 588  ☑ 588 

--------------------------------------------------
Iteration 73
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 4s 83us/step - loss: 0.0021 - acc: 0.9999 - val_loss: 0.0045 - val_acc: 0.9990
Q 94+5    T 99   ☑ 99  
Q 494+45  T 539  ☑ 539 
Q 519+444 T 963  ☑ 963 
Q 47+392  T 439  ☑ 439 
Q 187+191 T 378  ☑ 378 
Q 96+450  T 546  ☑ 546 
Q 1+56    T 57   ☑ 57  
Q 338+44  T 382  ☑ 382 
Q 634+70  T 704  ☑ 704 
Q 79+997  T 1076 ☑ 1076

--------------------------------------------------
Iteration 86
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 83us/step - loss: 0.0020 - acc: 0.9999 - val_loss: 0.0044 - val_acc: 0.9990
Q 416+3   T 419  ☑ 419 
Q 688+92  T 780  ☑ 780 
Q 837+878 T 1715 ☑ 1715
Q 376+128 T 504  ☑ 504 
Q 525+700 T 1225 ☑ 1225
Q 928+7   T 935  ☑ 935 
Q 8+812   T 820  ☑ 820 
Q 743+7   T 750  ☑ 750 
Q 291+77  T 368  ☑ 368 
Q 45+951  T 996  ☑ 996 

--------------------------------------------------
Iteration 87
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

45000/45000 [==============================] - 4s 83us/step - loss: 0.0017 - acc: 0.9999 - val_loss: 0.0045 - val_acc: 0.9989
Q 7+63    T 70   ☑ 70  
Q 33+967  T 1000 ☑ 1000
Q 506+3   T 509  ☑ 509 
Q 67+475  T 542  ☑ 542 
Q 314+11  T 325  ☑ 325 
Q 89+61   T 150  ☑ 150 
Q 14+591  T 605  ☑ 605 
Q 526+718 T 1244 ☑ 1244
Q 11+68   T 79   ☑ 79  
Q 23+273  T 296  ☑ 296 


In [31]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 4+68    T 72   ☑ 72  
Q 1+55    T 56   ☑ 56  
Q 3+0     T 3    ☑ 3   
Q 497+2   T 499  ☑ 499 
Q 23+503  T 526  ☑ 526 
Q 692+506 T 1198 ☑ 1198
Q 67+6    T 73   ☑ 73  
Q 1+88    T 89   ☑ 89  
Q 52+405  T 457  ☑ 457 
Q 49+2    T 51   ☑ 51  
Testing Accuracy :  0.98


### 1.3 實驗使用不同的epoch和batch size訓練
* with batch = 128, epoch = 200

In [32]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [33]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 207us/step - loss: 1.8880 - acc: 0.3216 - val_loss: 1.7872 - val_acc: 0.3370
Q 95+526  T 621  ☒ 100 
Q 500+5   T 505  ☒ 351 
Q 74+376  T 450  ☒ 101 
Q 50+735  T 785  ☒ 101 
Q 288+0   T 288  ☒ 320 
Q 811+57  T 868  ☒ 101 
Q 3+465   T 468  ☒ 33  
Q 291+77  T 368  ☒ 101 
Q 879+991 T 1870 ☒ 1111
Q 962+20  T 982  ☒ 101 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 1.7363 - acc: 0.3590 - val_loss: 1.6747 - val_acc: 0.3755
Q 342+86  T 428  ☒ 386 
Q 1+538   T 539  ☒ 13  
Q 363+9   T 372  ☒ 336 
Q 866+83  T 949  ☒ 803 
Q 803+72  T 875  ☒ 883 
Q 659+7   T 666  ☒ 106 
Q 79+131  T 210  ☒ 101 
Q 33+612  T 645  ☒ 333 
Q 974+55  T 1029 ☒ 903 
Q 452+61  T 513  ☒ 366 

-------------------

45000/45000 [==============================] - 7s 165us/step - loss: 0.1399 - acc: 0.9744 - val_loss: 0.1253 - val_acc: 0.9746
Q 52+398  T 450  ☒ 440 
Q 39+46   T 85   ☑ 85  
Q 792+3   T 795  ☑ 795 
Q 96+30   T 126  ☑ 126 
Q 84+308  T 392  ☑ 392 
Q 206+407 T 613  ☑ 613 
Q 9+533   T 542  ☑ 542 
Q 196+0   T 196  ☑ 196 
Q 28+775  T 803  ☑ 803 
Q 66+63   T 129  ☑ 129 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 0.1086 - acc: 0.9807 - val_loss: 0.0974 - val_acc: 0.9815
Q 833+48  T 881  ☑ 881 
Q 270+145 T 415  ☑ 415 
Q 781+60  T 841  ☑ 841 
Q 44+43   T 87   ☑ 87  
Q 998+12  T 1010 ☑ 1010
Q 742+514 T 1256 ☑ 1256
Q 77+74   T 151  ☑ 151 
Q 427+539 T 966  ☑ 966 
Q 588+92  T 680  ☒ 670 
Q 639+99  T 738  ☑ 738 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 161us/step - loss: 0.0533 - acc: 0.9838 - val_loss: 0.0627 - val_acc: 0.9764
Q 82+854  T 936  ☑ 936 
Q 552+73  T 625  ☑ 625 
Q 804+1   T 805  ☑ 805 
Q 522+5   T 527  ☑ 527 
Q 656+33  T 689  ☑ 689 
Q 688+35  T 723  ☑ 723 
Q 51+891  T 942  ☑ 942 
Q 513+88  T 601  ☑ 601 
Q 818+4   T 822  ☑ 822 
Q 622+873 T 1495 ☑ 1495

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 0.0162 - acc: 0.9968 - val_loss: 0.0110 - val_acc: 0.9984
Q 4+619   T 623  ☑ 623 
Q 391+671 T 1062 ☑ 1062
Q 799+94  T 893  ☑ 893 
Q 812+83  T 895  ☑ 895 
Q 726+50  T 776  ☑ 776 
Q 940+77  T 1017 ☑ 1017
Q 832+920 T 1752 ☑ 1752
Q 801+972 T 1773 ☑ 1773
Q 706+110 T 816  ☑ 816 
Q 20+707  T 727  ☑ 727 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 158us/step - loss: 0.0028 - acc: 0.9997 - val_loss: 0.0134 - val_acc: 0.9963
Q 221+987 T 1208 ☑ 1208
Q 519+288 T 807  ☑ 807 
Q 7+607   T 614  ☑ 614 
Q 286+79  T 365  ☑ 365 
Q 62+63   T 125  ☑ 125 
Q 668+22  T 690  ☑ 690 
Q 55+980  T 1035 ☑ 1035
Q 42+72   T 114  ☑ 114 
Q 76+4    T 80   ☑ 80  
Q 232+53  T 285  ☑ 285 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 0.0498 - acc: 0.9852 - val_loss: 0.0080 - val_acc: 0.9983
Q 891+237 T 1128 ☑ 1128
Q 68+738  T 806  ☑ 806 
Q 425+6   T 431  ☑ 431 
Q 94+294  T 388  ☑ 388 
Q 88+953  T 1041 ☑ 1041
Q 18+376  T 394  ☑ 394 
Q 54+268  T 322  ☑ 322 
Q 3+64    T 67   ☑ 67  
Q 389+468 T 857  ☑ 857 
Q 386+93  T 479  ☑ 479 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 158us/step - loss: 0.0075 - acc: 0.9983 - val_loss: 0.0085 - val_acc: 0.9979
Q 894+52  T 946  ☑ 946 
Q 97+896  T 993  ☑ 993 
Q 97+419  T 516  ☑ 516 
Q 725+14  T 739  ☑ 739 
Q 71+864  T 935  ☑ 935 
Q 857+743 T 1600 ☑ 1600
Q 96+307  T 403  ☑ 403 
Q 88+953  T 1041 ☑ 1041
Q 554+172 T 726  ☑ 726 
Q 93+600  T 693  ☑ 693 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 157us/step - loss: 0.0021 - acc: 0.9998 - val_loss: 0.0025 - val_acc: 0.9997
Q 554+91  T 645  ☑ 645 
Q 391+424 T 815  ☑ 815 
Q 1+247   T 248  ☑ 248 
Q 491+85  T 576  ☑ 576 
Q 99+499  T 598  ☑ 598 
Q 141+293 T 434  ☑ 434 
Q 324+57  T 381  ☑ 381 
Q 915+374 T 1289 ☑ 1289
Q 42+319  T 361  ☑ 361 
Q 684+308 T 992  ☑ 992 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

...

45000/45000 [==============================] - 7s 166us/step - loss: 3.7341e-04 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9998
Q 42+737  T 779  ☑ 779 
Q 56+999  T 1055 ☑ 1055
Q 3+558   T 561  ☑ 561 
Q 73+829  T 902  ☑ 902 
Q 935+9   T 944  ☑ 944 
Q 28+968  T 996  ☑ 996 
Q 291+9   T 300  ☑ 300 
Q 26+653  T 679  ☑ 679 
Q 1+251   T 252  ☑ 252 
Q 722+218 T 940  ☑ 940 

--------------------------------------------------
Iteration 142
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 165us/step - loss: 2.9511e-04 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 0.9998
Q 2+850   T 852  ☑ 852 
Q 62+63   T 125  ☑ 125 
Q 458+234 T 692  ☑ 692 
Q 335+810 T 1145 ☑ 1145
Q 83+232  T 315  ☑ 315 
Q 459+191 T 650  ☑ 650 
Q 948+605 T 1553 ☑ 1553
Q 193+695 T 888  ☑ 888 
Q 300+7   T 307  ☑ 307 
Q 480+760 T 1240 ☑ 1240

--------------------------------------------------
Iteration 143
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45

45000/45000 [==============================] - 7s 161us/step - loss: 2.0616e-04 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 0.9998
Q 608+1   T 609  ☑ 609 
Q 235+540 T 775  ☑ 775 
Q 86+868  T 954  ☑ 954 
Q 191+44  T 235  ☑ 235 
Q 571+77  T 648  ☑ 648 
Q 790+639 T 1429 ☑ 1429
Q 725+14  T 739  ☑ 739 
Q 12+13   T 25   ☑ 25  
Q 44+211  T 255  ☑ 255 
Q 182+696 T 878  ☑ 878 

--------------------------------------------------
Iteration 156
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 165us/step - loss: 1.7737e-04 - acc: 1.0000 - val_loss: 9.9931e-04 - val_acc: 0.9998
Q 551+41  T 592  ☑ 592 
Q 509+34  T 543  ☑ 543 
Q 928+270 T 1198 ☑ 1198
Q 535+5   T 540  ☑ 540 
Q 73+47   T 120  ☑ 120 
Q 93+612  T 705  ☑ 705 
Q 701+269 T 970  ☑ 970 
Q 93+14   T 107  ☑ 107 
Q 459+191 T 650  ☑ 650 
Q 303+73  T 376  ☑ 376 

--------------------------------------------------
Iteration 157
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
4500

45000/45000 [==============================] - 7s 161us/step - loss: 4.2259e-04 - acc: 1.0000 - val_loss: 0.0010 - val_acc: 0.9999
Q 876+28  T 904  ☑ 904 
Q 23+273  T 296  ☑ 296 
Q 988+894 T 1882 ☑ 1882
Q 859+306 T 1165 ☑ 1165
Q 677+0   T 677  ☑ 677 
Q 37+863  T 900  ☑ 900 
Q 409+728 T 1137 ☑ 1137
Q 123+657 T 780  ☑ 780 
Q 709+95  T 804  ☑ 804 
Q 437+974 T 1411 ☑ 1411

--------------------------------------------------
Iteration 170
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 2.6530e-04 - acc: 1.0000 - val_loss: 8.7616e-04 - val_acc: 0.9998
Q 388+73  T 461  ☑ 461 
Q 471+704 T 1175 ☑ 1175
Q 16+594  T 610  ☑ 610 
Q 832+145 T 977  ☑ 977 
Q 33+921  T 954  ☑ 954 
Q 174+812 T 986  ☑ 986 
Q 160+876 T 1036 ☑ 1036
Q 1+855   T 856  ☑ 856 
Q 799+579 T 1378 ☑ 1378
Q 36+99   T 135  ☑ 135 

--------------------------------------------------
Iteration 171
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
4500

45000/45000 [==============================] - 7s 161us/step - loss: 1.4618e-04 - acc: 1.0000 - val_loss: 7.3633e-04 - val_acc: 0.9999
Q 66+9    T 75   ☑ 75  
Q 8+799   T 807  ☑ 807 
Q 218+301 T 519  ☑ 519 
Q 81+980  T 1061 ☑ 1061
Q 822+25  T 847  ☑ 847 
Q 4+875   T 879  ☑ 879 
Q 901+38  T 939  ☑ 939 
Q 445+613 T 1058 ☑ 1058
Q 91+512  T 603  ☑ 603 
Q 570+9   T 579  ☑ 579 

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 161us/step - loss: 1.3024e-04 - acc: 1.0000 - val_loss: 6.8174e-04 - val_acc: 0.9998
Q 344+8   T 352  ☑ 352 
Q 68+285  T 353  ☑ 353 
Q 109+2   T 111  ☑ 111 
Q 34+267  T 301  ☑ 301 
Q 696+62  T 758  ☑ 758 
Q 89+617  T 706  ☑ 706 
Q 458+0   T 458  ☑ 458 
Q 42+493  T 535  ☑ 535 
Q 14+474  T 488  ☑ 488 
Q 34+820  T 854  ☑ 854 

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1


45000/45000 [==============================] - 7s 161us/step - loss: 9.2379e-05 - acc: 1.0000 - val_loss: 6.6210e-04 - val_acc: 0.9998
Q 30+208  T 238  ☑ 238 
Q 706+110 T 816  ☑ 816 
Q 415+152 T 567  ☑ 567 
Q 50+452  T 502  ☑ 502 
Q 50+445  T 495  ☑ 495 
Q 548+101 T 649  ☑ 649 
Q 541+812 T 1353 ☑ 1353
Q 964+47  T 1011 ☑ 1011
Q 828+58  T 886  ☑ 886 
Q 54+953  T 1007 ☑ 1007

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 162us/step - loss: 0.0249 - acc: 0.9929 - val_loss: 0.0034 - val_acc: 0.9991
Q 536+876 T 1412 ☑ 1412
Q 702+57  T 759  ☑ 759 
Q 878+90  T 968  ☑ 968 
Q 113+791 T 904  ☑ 904 
Q 23+360  T 383  ☑ 383 
Q 51+370  T 421  ☑ 421 
Q 323+910 T 1233 ☑ 1233
Q 863+357 T 1220 ☑ 1220
Q 26+21   T 47   ☑ 47  
Q 74+5    T 79   ☑ 79  

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45

In [34]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 4+68    T 72   ☑ 72  
Q 1+55    T 56   ☑ 56  
Q 3+0     T 3    ☑ 3   
Q 497+2   T 499  ☑ 499 
Q 23+503  T 526  ☑ 526 
Q 692+506 T 1198 ☑ 1198
Q 67+6    T 73   ☑ 73  
Q 1+88    T 89   ☑ 89  
Q 52+405  T 457  ☑ 457 
Q 49+2    T 51   ☑ 51  
Testing Accuracy :  0.991


## 2. 實驗不同位數的數字
* The digits of input number = 4

In [35]:
# Generate different data

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 4
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)
print(ctable)

questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 9, 12)
(45000, 5, 12)
Validation Data:
(5000, 9, 12)
(5000, 5, 12)


In [36]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 5, 128)            131584    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 5, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [37]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 242us/step - loss: 1.7990 - acc: 0.3520 - val_loss: 1.6663 - val_acc: 0.3846
Q 2259+73   T 2332  ☒ 1233 
Q 4746+79   T 4825  ☒ 1776 
Q 4+5614    T 5618  ☒ 1666 
Q 81+7670   T 7751  ☒ 1777 
Q 0+27      T 27    ☒ 12   
Q 871+116   T 987   ☒ 177  
Q 8670+65   T 8735  ☒ 1766 
Q 778+843   T 1621  ☒ 1777 
Q 96+539    T 635   ☒ 166  
Q 1330+8966 T 10296 ☒ 1133 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 194us/step - loss: 1.6009 - acc: 0.4077 - val_loss: 1.5410 - val_acc: 0.4321
Q 931+92    T 1023  ☒ 9003 
Q 5118+16   T 5134  ☒ 1116 
Q 993+6836  T 7829  ☒ 1066 
Q 557+6220  T 6777  ☒ 1566 
Q 0+7472    T 7472  ☒ 7778 
Q 5793+66   T 5859  ☒ 9666 
Q 3705+107  T 3812  ☒ 1016 
Q 2+9682    T 9684  ☒ 9

45000/45000 [==============================] - 8s 187us/step - loss: 0.5136 - acc: 0.8145 - val_loss: 0.4959 - val_acc: 0.8230
Q 223+9235  T 9458  ☒ 9455 
Q 8471+3    T 8474  ☑ 8474 
Q 508+4559  T 5067  ☒ 5078 
Q 66+7905   T 7971  ☒ 7972 
Q 1+737     T 738   ☑ 738  
Q 747+3293  T 4040  ☒ 3059 
Q 2+4474    T 4476  ☑ 4476 
Q 5885+9    T 5894  ☑ 5894 
Q 94+534    T 628   ☑ 628  
Q 540+1307  T 1847  ☒ 1766 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 197us/step - loss: 0.4655 - acc: 0.8344 - val_loss: 0.4590 - val_acc: 0.8320
Q 168+73    T 241   ☑ 241  
Q 708+6383  T 7091  ☒ 7080 
Q 29+97     T 126   ☒ 125  
Q 393+3336  T 3729  ☒ 3722 
Q 53+8785   T 8838  ☒ 8839 
Q 79+792    T 871   ☑ 871  
Q 204+3308  T 3512  ☒ 3420 
Q 21+8593   T 8614  ☑ 8614 
Q 5817+3309 T 9126  ☒ 9000 
Q 11+920    T 931   ☒ 923  

--------------------------------------------------
Iteration 

Q 25+5577   T 5602  ☑ 5602 
Q 54+1344   T 1398  ☒ 1399 
Q 9966+1758 T 11724 ☒ 11733
Q 351+7754  T 8105  ☒ 8115 
Q 448+966   T 1414  ☒ 1413 
Q 481+211   T 692   ☑ 692  
Q 4754+8    T 4762  ☑ 4762 
Q 4752+706  T 5458  ☑ 5458 
Q 56+790    T 846   ☑ 846  
Q 243+18    T 261   ☑ 261  

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 192us/step - loss: 0.1126 - acc: 0.9670 - val_loss: 0.1357 - val_acc: 0.9561
Q 6695+90   T 6785  ☒ 6775 
Q 4881+141  T 5022  ☑ 5022 
Q 325+2     T 327   ☑ 327  
Q 368+2129  T 2497  ☒ 2597 
Q 5+607     T 612   ☑ 612  
Q 685+1808  T 2493  ☑ 2493 
Q 5+2909    T 2914  ☑ 2914 
Q 146+2     T 148   ☑ 148  
Q 48+253    T 301   ☑ 301  
Q 2+199     T 201   ☑ 201  

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 194us/step - lo

45000/45000 [==============================] - 9s 193us/step - loss: 0.0408 - acc: 0.9893 - val_loss: 0.0628 - val_acc: 0.9786
Q 7+794     T 801   ☑ 801  
Q 925+423   T 1348  ☑ 1348 
Q 309+6     T 315   ☑ 315  
Q 9+2882    T 2891  ☑ 2891 
Q 3888+14   T 3902  ☒ 3901 
Q 6+3684    T 3690  ☑ 3690 
Q 2060+39   T 2099  ☑ 2099 
Q 4+4133    T 4137  ☑ 4137 
Q 3521+11   T 3532  ☑ 3532 
Q 21+5468   T 5489  ☑ 5489 

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 193us/step - loss: 0.0284 - acc: 0.9941 - val_loss: 0.0644 - val_acc: 0.9790
Q 643+3     T 646   ☑ 646  
Q 26+1452   T 1478  ☑ 1478 
Q 42+6363   T 6405  ☑ 6405 
Q 4440+7    T 4447  ☑ 4447 
Q 892+36    T 928   ☑ 928  
Q 235+3675  T 3910  ☑ 3910 
Q 8+2309    T 2317  ☑ 2317 
Q 9387+28   T 9415  ☑ 9415 
Q 3416+6281 T 9697  ☑ 9697 
Q 7727+220  T 7947  ☑ 7947 

--------------------------------------------------
Iteration 

Q 9243+51   T 9294  ☒ 9204 
Q 866+7258  T 8124  ☑ 8124 
Q 8986+4    T 8990  ☑ 8990 
Q 117+66    T 183   ☑ 183  
Q 879+264   T 1143  ☑ 1143 
Q 94+765    T 859   ☑ 859  
Q 162+688   T 850   ☑ 850  
Q 347+193   T 540   ☑ 540  
Q 3141+380  T 3521  ☑ 3521 
Q 9+701     T 710   ☑ 710  

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 192us/step - loss: 0.0278 - acc: 0.9924 - val_loss: 0.0455 - val_acc: 0.9855
Q 1357+307  T 1664  ☑ 1664 
Q 775+9149  T 9924  ☑ 9924 
Q 933+818   T 1751  ☑ 1751 
Q 3371+1027 T 4398  ☑ 4398 
Q 616+83    T 699   ☑ 699  
Q 879+74    T 953   ☑ 953  
Q 844+673   T 1517  ☑ 1517 
Q 84+296    T 380   ☑ 380  
Q 9+8170    T 8179  ☑ 8179 
Q 91+197    T 288   ☑ 288  

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 192us/step - lo

...

45000/45000 [==============================] - 8s 187us/step - loss: 4.4956e-04 - acc: 1.0000 - val_loss: 0.0190 - val_acc: 0.9939
Q 87+4490   T 4577  ☑ 4577 
Q 706+4593  T 5299  ☑ 5299 
Q 6+1217    T 1223  ☑ 1223 
Q 5817+3309 T 9126  ☑ 9126 
Q 5932+85   T 6017  ☑ 6017 
Q 816+534   T 1350  ☑ 1350 
Q 793+5435  T 6228  ☑ 6228 
Q 3515+6926 T 10441 ☑ 10441
Q 385+958   T 1343  ☑ 1343 
Q 406+6699  T 7105  ☑ 7105 

--------------------------------------------------
Iteration 135
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 4.1039e-04 - acc: 1.0000 - val_loss: 0.0190 - val_acc: 0.9938
Q 8185+1    T 8186  ☑ 8186 
Q 3430+6527 T 9957  ☑ 9957 
Q 7084+6    T 7090  ☑ 7090 
Q 417+5     T 422   ☑ 422  
Q 949+45    T 994   ☑ 994  
Q 2319+118  T 2437  ☑ 2437 
Q 56+790    T 846   ☑ 846  
Q 1809+5996 T 7805  ☑ 7805 
Q 879+1888  T 2767  ☑ 2767 
Q 419+54    T 473   ☑ 473  

--------------------------------------------------
I

45000/45000 [==============================] - 9s 193us/step - loss: 8.3954e-04 - acc: 1.0000 - val_loss: 0.0209 - val_acc: 0.9935
Q 7604+8    T 7612  ☑ 7612 
Q 1012+89   T 1101  ☑ 1101 
Q 718+9     T 727   ☑ 727  
Q 3976+8228 T 12204 ☑ 12204
Q 7372+2    T 7374  ☑ 7374 
Q 80+812    T 892   ☑ 892  
Q 6760+1253 T 8013  ☑ 8013 
Q 17+5394   T 5411  ☑ 5411 
Q 4+744     T 748   ☑ 748  
Q 30+972    T 1002  ☑ 1002 

--------------------------------------------------
Iteration 148
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 194us/step - loss: 5.9374e-04 - acc: 1.0000 - val_loss: 0.0204 - val_acc: 0.9934
Q 44+661    T 705   ☑ 705  
Q 4+15      T 19    ☑ 19   
Q 4693+4481 T 9174  ☑ 9174 
Q 96+4183   T 4279  ☑ 4279 
Q 815+8901  T 9716  ☑ 9716 
Q 9+255     T 264   ☑ 264  
Q 3+9428    T 9431  ☑ 9431 
Q 963+806   T 1769  ☒ 1779 
Q 1300+4    T 1304  ☑ 1304 
Q 416+838   T 1254  ☑ 1254 

--------------------------------------------------
I

45000/45000 [==============================] - 9s 191us/step - loss: 0.0335 - acc: 0.9919 - val_loss: 0.2849 - val_acc: 0.9240
Q 90+13     T 103   ☑ 103  
Q 2571+39   T 2610  ☑ 2610 
Q 5033+8    T 5041  ☒ 5031 
Q 1+352     T 353   ☑ 353  
Q 220+7577  T 7797  ☒ 7787 
Q 80+31     T 111   ☑ 111  
Q 6+8169    T 8175  ☑ 8175 
Q 353+9     T 362   ☑ 362  
Q 48+253    T 301   ☑ 301  
Q 489+4     T 493   ☑ 493  

--------------------------------------------------
Iteration 161
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 193us/step - loss: 0.0447 - acc: 0.9858 - val_loss: 0.0395 - val_acc: 0.9870
Q 4883+82   T 4965  ☑ 4965 
Q 3+602     T 605   ☒ 606  
Q 3+993     T 996   ☑ 996  
Q 821+0     T 821   ☑ 821  
Q 662+673   T 1335  ☑ 1335 
Q 78+240    T 318   ☑ 318  
Q 7837+49   T 7886  ☑ 7886 
Q 659+6081  T 6740  ☑ 6740 
Q 75+3220   T 3295  ☑ 3295 
Q 1319+3923 T 5242  ☑ 5242 

--------------------------------------------------
Iteration

Q 4789+54   T 4843  ☑ 4843 
Q 8203+884  T 9087  ☑ 9087 
Q 13+830    T 843   ☑ 843  
Q 4356+4    T 4360  ☑ 4360 
Q 96+3392   T 3488  ☑ 3488 
Q 32+674    T 706   ☑ 706  
Q 548+23    T 571   ☑ 571  
Q 542+836   T 1378  ☑ 1378 
Q 3+90      T 93    ☑ 93   
Q 1575+7222 T 8797  ☑ 8797 

--------------------------------------------------
Iteration 174
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 187us/step - loss: 4.8110e-04 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 0.9944
Q 682+7     T 689   ☑ 689  
Q 655+5976  T 6631  ☑ 6631 
Q 6366+0    T 6366  ☑ 6366 
Q 975+3     T 978   ☑ 978  
Q 78+240    T 318   ☑ 318  
Q 784+86    T 870   ☑ 870  
Q 91+205    T 296   ☑ 296  
Q 7860+3918 T 11778 ☑ 11778
Q 2568+3    T 2571  ☑ 2571 
Q 9946+4    T 9950  ☑ 9950 

--------------------------------------------------
Iteration 175
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 189us/ste

45000/45000 [==============================] - 8s 187us/step - loss: 1.2571e-04 - acc: 1.0000 - val_loss: 0.0165 - val_acc: 0.9953
Q 386+9274  T 9660  ☑ 9660 
Q 43+821    T 864   ☑ 864  
Q 2270+851  T 3121  ☑ 3121 
Q 636+3595  T 4231  ☑ 4231 
Q 6825+465  T 7290  ☑ 7290 
Q 557+2703  T 3260  ☑ 3260 
Q 4490+629  T 5119  ☑ 5119 
Q 7568+88   T 7656  ☑ 7656 
Q 8203+986  T 9189  ☑ 9189 
Q 9573+117  T 9690  ☑ 9690 

--------------------------------------------------
Iteration 188
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 188us/step - loss: 1.1409e-04 - acc: 1.0000 - val_loss: 0.0163 - val_acc: 0.9953
Q 274+497   T 771   ☑ 771  
Q 301+3     T 304   ☑ 304  
Q 329+723   T 1052  ☑ 1052 
Q 7611+78   T 7689  ☑ 7689 
Q 789+44    T 833   ☑ 833  
Q 450+9     T 459   ☑ 459  
Q 42+79     T 121   ☑ 121  
Q 308+63    T 371   ☑ 371  
Q 9059+1    T 9060  ☑ 9060 
Q 3+1530    T 1533  ☑ 1533 

--------------------------------------------------
I

In [38]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [39]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 6+3668    T 3674  ☑ 3674 
Q 7+361     T 368   ☑ 368  
Q 441+792   T 1233  ☑ 1233 
Q 8946+389  T 9335  ☑ 9335 
Q 521+477   T 998   ☑ 998  
Q 8+427     T 435   ☑ 435  
Q 2637+3195 T 5832  ☑ 5832 
Q 9+1       T 10    ☑ 10   
Q 7+6       T 13    ☑ 13   
Q 915+65    T 980   ☑ 980  
Testing Accuracy :  0.982


## 3. Add more number
* Add 3 number (3 digits) together

In [40]:
def Generate_OtherData(DIGITS = 3, DATA_SIZE = 50000):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b, c = f(), f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b, c)))
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}+{}'.format(a, b, c)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b + c)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        expected.append(ans)
    print('Total addition questions:', len(questions))
    
    return questions, expected

In [41]:
# Generate different data

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)
print(ctable)

questions, expected = Generate_OtherData(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 11, 12)
(45000, 4, 12)
Validation Data:
(5000, 11, 12)
(5000, 4, 12)


In [46]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 4, 128)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [47]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 262us/step - loss: 1.8691 - acc: 0.3476 - val_loss: 1.7813 - val_acc: 0.3696
Q 81+9+25     T 115  ☒ 103 
Q 154+4+1     T 159  ☒ 10  
Q 48+96+250   T 394  ☒ 100 
Q 0+8+239     T 247  ☒ 103 
Q 849+5+6     T 860  ☒ 103 
Q 325+46+17   T 388  ☒ 103 
Q 875+121+45  T 1041 ☒ 101 
Q 3+4+378     T 385  ☒ 10  
Q 4+751+2     T 757  ☒ 10  
Q 500+290+37  T 827  ☒ 100 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 206us/step - loss: 1.7803 - acc: 0.3687 - val_loss: 1.7648 - val_acc: 0.3718
Q 82+537+221  T 840  ☒ 133 
Q 55+9+9      T 73   ☒ 10  
Q 313+4+512   T 829  ☒ 135 
Q 4+31+38     T 73   ☒ 11  
Q 77+246+571  T 894  ☒ 115 
Q 49+201+413  T 663  ☒ 135 
Q 60+35+320   T 415  ☒ 135 
Q 43+7+69     T 119  ☒ 

45000/45000 [==============================] - 9s 198us/step - loss: 0.9170 - acc: 0.6570 - val_loss: 0.9156 - val_acc: 0.6560
Q 9+996+18    T 1023 ☒ 1018
Q 842+7+424   T 1273 ☒ 1288
Q 19+44+5     T 68   ☑ 68  
Q 503+36+0    T 539  ☒ 544 
Q 4+129+4     T 137  ☒ 143 
Q 85+96+2     T 183  ☒ 189 
Q 641+331+0   T 972  ☒ 998 
Q 31+59+41    T 131  ☒ 128 
Q 8+304+0     T 312  ☒ 318 
Q 0+88+797    T 885  ☒ 879 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 197us/step - loss: 0.8877 - acc: 0.6705 - val_loss: 0.8872 - val_acc: 0.6618
Q 4+38+96     T 138  ☒ 147 
Q 214+49+807  T 1070 ☒ 1098
Q 6+448+7     T 461  ☒ 464 
Q 25+91+71    T 187  ☒ 198 
Q 535+7+883   T 1425 ☒ 1411
Q 2+724+473   T 1199 ☒ 1105
Q 490+9+2     T 501  ☒ 503 
Q 82+679+29   T 790  ☒ 783 
Q 0+30+8      T 38   ☒ 40  
Q 94+0+59     T 153  ☒ 151 

--------------------------------------------------
Iteration 

45000/45000 [==============================] - 9s 204us/step - loss: 0.6687 - acc: 0.7519 - val_loss: 0.6942 - val_acc: 0.7367
Q 143+15+565  T 723  ☒ 720 
Q 34+522+45   T 601  ☒ 501 
Q 390+4+72    T 466  ☒ 467 
Q 855+48+744  T 1647 ☒ 1645
Q 8+836+508   T 1352 ☒ 1349
Q 16+0+9      T 25   ☑ 25  
Q 6+80+75     T 161  ☒ 167 
Q 720+408+29  T 1157 ☒ 1152
Q 43+51+4     T 98   ☒ 90  
Q 70+92+515   T 677  ☑ 677 

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 0.6570 - acc: 0.7576 - val_loss: 0.7000 - val_acc: 0.7323
Q 81+9+505    T 595  ☒ 694 
Q 62+398+70   T 530  ☒ 537 
Q 170+5+24    T 199  ☒ 195 
Q 69+24+704   T 797  ☒ 802 
Q 2+5+65      T 72   ☑ 72  
Q 4+98+262    T 364  ☒ 360 
Q 411+35+11   T 457  ☑ 457 
Q 379+795+832 T 2006 ☒ 2010
Q 61+76+244   T 381  ☒ 382 
Q 0+21+2      T 23   ☑ 23  

--------------------------------------------------
Iteration 

Q 65+156+11   T 232  ☒ 237 
Q 5+4+802     T 811  ☑ 811 
Q 2+37+143    T 182  ☒ 187 
Q 40+0+2      T 42   ☒ 44  
Q 456+9+3     T 468  ☒ 469 
Q 2+61+9      T 72   ☑ 72  
Q 5+266+820   T 1091 ☒ 1086
Q 190+33+781  T 1004 ☒ 1001
Q 3+73+18     T 94   ☒ 96  
Q 62+17+8     T 87   ☒ 83  

--------------------------------------------------
Iteration 41
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 207us/step - loss: 0.5775 - acc: 0.7875 - val_loss: 0.6407 - val_acc: 0.7555
Q 7+496+192   T 695  ☒ 799 
Q 3+694+4     T 701  ☑ 701 
Q 618+7+74    T 699  ☑ 699 
Q 35+30+234   T 299  ☒ 399 
Q 97+866+31   T 994  ☒ 990 
Q 652+96+604  T 1352 ☒ 1344
Q 278+498+33  T 809  ☒ 801 
Q 34+94+266   T 394  ☒ 391 
Q 343+2+41    T 386  ☒ 384 
Q 914+9+789   T 1712 ☒ 1711

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 203us/step - lo

45000/45000 [==============================] - 9s 200us/step - loss: 0.4964 - acc: 0.8191 - val_loss: 0.5886 - val_acc: 0.7781
Q 98+606+267  T 971  ☒ 970 
Q 284+1+9     T 294  ☒ 293 
Q 978+4+1     T 983  ☒ 982 
Q 26+98+88    T 212  ☒ 214 
Q 71+961+72   T 1104 ☒ 1106
Q 805+4+85    T 894  ☑ 894 
Q 8+766+3     T 777  ☑ 777 
Q 9+2+628     T 639  ☒ 649 
Q 18+72+535   T 625  ☒ 627 
Q 90+360+791  T 1241 ☒ 1244

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 0.4867 - acc: 0.8225 - val_loss: 0.5858 - val_acc: 0.7796
Q 96+671+27   T 794  ☒ 893 
Q 0+898+53    T 951  ☒ 949 
Q 31+77+623   T 731  ☒ 733 
Q 69+74+75    T 218  ☒ 216 
Q 4+990+7     T 1001 ☑ 1001
Q 8+943+416   T 1367 ☒ 1366
Q 71+5+0      T 76   ☑ 76  
Q 2+1+434     T 437  ☑ 437 
Q 888+50+6    T 944  ☒ 941 
Q 831+58+379  T 1268 ☒ 1270

--------------------------------------------------
Iteration 

...

45000/45000 [==============================] - 9s 200us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0260 - val_acc: 0.9907
Q 404+93+7    T 504  ☑ 504 
Q 356+201+38  T 595  ☑ 595 
Q 84+4+50     T 138  ☑ 138 
Q 6+29+91     T 126  ☑ 126 
Q 92+98+2     T 192  ☑ 192 
Q 62+420+47   T 529  ☑ 529 
Q 77+815+2    T 894  ☑ 894 
Q 43+653+1    T 697  ☑ 697 
Q 180+3+697   T 880  ☑ 880 
Q 732+61+99   T 892  ☑ 892 

--------------------------------------------------
Iteration 135
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0261 - val_acc: 0.9911
Q 33+636+98   T 767  ☑ 767 
Q 3+824+7     T 834  ☑ 834 
Q 716+10+718  T 1444 ☑ 1444
Q 5+498+95    T 598  ☑ 598 
Q 6+17+1      T 24   ☑ 24  
Q 46+743+25   T 814  ☑ 814 
Q 69+76+549   T 694  ☑ 694 
Q 3+709+8     T 720  ☑ 720 
Q 2+8+792     T 802  ☑ 802 
Q 6+24+68     T 98   ☑ 98  

--------------------------------------------------
Iteration

Q 116+960+4   T 1080 ☒ 1070
Q 965+13+38   T 1016 ☑ 1016
Q 41+133+6    T 180  ☑ 180 
Q 690+9+1     T 700  ☑ 700 
Q 436+615+918 T 1969 ☑ 1969
Q 7+252+460   T 719  ☑ 719 
Q 74+51+2     T 127  ☑ 127 
Q 71+730+544  T 1345 ☑ 1345
Q 7+643+505   T 1155 ☑ 1155
Q 3+78+5      T 86   ☑ 86  

--------------------------------------------------
Iteration 148
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - loss: 0.0034 - acc: 0.9997 - val_loss: 0.0251 - val_acc: 0.9919
Q 0+6+443     T 449  ☑ 449 
Q 945+56+842  T 1843 ☑ 1843
Q 148+34+1    T 183  ☑ 183 
Q 1+729+81    T 811  ☑ 811 
Q 99+596+55   T 750  ☑ 750 
Q 54+6+93     T 153  ☑ 153 
Q 36+1+9      T 46   ☑ 46  
Q 694+904+1   T 1599 ☒ 1699
Q 596+2+81    T 679  ☑ 679 
Q 133+15+79   T 227  ☑ 227 

--------------------------------------------------
Iteration 149
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - 

45000/45000 [==============================] - 9s 200us/step - loss: 0.0176 - acc: 0.9949 - val_loss: 0.0344 - val_acc: 0.9886
Q 856+94+966  T 1916 ☑ 1916
Q 6+92+8      T 106  ☑ 106 
Q 98+5+450    T 553  ☑ 553 
Q 478+22+0    T 500  ☑ 500 
Q 808+738+4   T 1550 ☑ 1550
Q 857+204+740 T 1801 ☑ 1801
Q 8+475+5     T 488  ☑ 488 
Q 382+320+729 T 1431 ☑ 1431
Q 7+23+43     T 73   ☑ 73  
Q 348+53+2    T 403  ☑ 403 

--------------------------------------------------
Iteration 162
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 0.0038 - acc: 0.9996 - val_loss: 0.0238 - val_acc: 0.9925
Q 1+989+8     T 998  ☑ 998 
Q 1+72+5      T 78   ☑ 78  
Q 213+9+2     T 224  ☑ 224 
Q 697+633+68  T 1398 ☑ 1398
Q 62+37+2     T 101  ☑ 101 
Q 3+6+105     T 114  ☑ 114 
Q 5+1+164     T 170  ☑ 170 
Q 13+4+7      T 24   ☑ 24  
Q 877+53+4    T 934  ☑ 934 
Q 118+901+472 T 1491 ☑ 1491

--------------------------------------------------
Iteration

Q 2+677+994   T 1673 ☒ 1672
Q 62+4+41     T 107  ☑ 107 
Q 8+110+39    T 157  ☑ 157 
Q 6+894+4     T 904  ☑ 904 
Q 3+4+311     T 318  ☑ 318 
Q 282+475+308 T 1065 ☑ 1065
Q 256+699+462 T 1417 ☒ 1327
Q 48+60+1     T 109  ☑ 109 
Q 5+326+30    T 361  ☑ 361 
Q 740+9+4     T 753  ☑ 753 

--------------------------------------------------
Iteration 175
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - loss: 0.0306 - acc: 0.9905 - val_loss: 0.0305 - val_acc: 0.9897
Q 8+491+2     T 501  ☑ 501 
Q 601+7+721   T 1329 ☑ 1329
Q 526+592+3   T 1121 ☑ 1121
Q 64+675+896  T 1635 ☒ 1645
Q 114+0+6     T 120  ☑ 120 
Q 3+62+6      T 71   ☑ 71  
Q 57+66+196   T 319  ☑ 319 
Q 0+7+66      T 73   ☑ 73  
Q 2+83+17     T 102  ☑ 102 
Q 2+759+1     T 762  ☑ 762 

--------------------------------------------------
Iteration 176
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - 

45000/45000 [==============================] - 9s 199us/step - loss: 8.5195e-04 - acc: 1.0000 - val_loss: 0.0174 - val_acc: 0.9943
Q 822+6+65    T 893  ☑ 893 
Q 75+4+77     T 156  ☑ 156 
Q 598+71+935  T 1604 ☑ 1604
Q 178+4+23    T 205  ☑ 205 
Q 28+3+799    T 830  ☑ 830 
Q 95+9+88     T 192  ☑ 192 
Q 9+69+240    T 318  ☑ 318 
Q 308+2+58    T 368  ☑ 368 
Q 20+8+543    T 571  ☑ 571 
Q 633+0+24    T 657  ☑ 657 

--------------------------------------------------
Iteration 189
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 8.3169e-04 - acc: 1.0000 - val_loss: 0.0175 - val_acc: 0.9944
Q 7+12+3      T 22   ☑ 22  
Q 39+5+278    T 322  ☑ 322 
Q 97+156+18   T 271  ☑ 271 
Q 544+73+200  T 817  ☑ 817 
Q 87+484+88   T 659  ☑ 659 
Q 8+31+36     T 75   ☑ 75  
Q 908+7+5     T 920  ☑ 920 
Q 71+866+6    T 943  ☑ 943 
Q 50+95+42    T 187  ☑ 187 
Q 1+6+2       T 9    ☑ 9   

--------------------------------------------------
I

In [48]:
## Generate Testing Data
questions_test, expected_test = Generate_OtherData(DIGITS = DIGITS, DATA_SIZE = 1000)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [49]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 68+996+5    T 1069 ☑ 1069
Q 123+236+240 T 599  ☑ 599 
Q 50+2+40     T 92   ☑ 92  
Q 7+593+82    T 682  ☑ 682 
Q 27+0+423    T 450  ☑ 450 
Q 7+3+74      T 84   ☑ 84  
Q 5+8+25      T 38   ☑ 38  
Q 59+35+65    T 159  ☑ 159 
Q 281+30+723  T 1034 ☑ 1034
Q 3+0+15      T 18   ☑ 18  
Testing Accuracy :  0.964


# 總結
## 在Addition_rnn中採用lstm實現seq2seq的加法器
## 實驗:
### 1. 採用四種不同的batch size和訓練epochs (兩個三位數相加)
###     * [ batch size = 128, epoch = 100 ] : Validation acc = 99.98% / Testing acc (1000 testing data) = 0.99
###     * [ batch size = 64,   epoch = 100 ] : Validation acc = 99.98% / Testing acc (1000 testing data) = 0.991
###     * [ batch size = 256, epoch = 100 ] : Validation acc = 99.94% / Testing acc (1000 testing data) = 0.98
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 99.98% / Testing acc (1000 testing data) = 0.991
### 2. 兩個四位數相加
###     * [ batch size = 128, epoch = 100 ] : Validation acc = 99.58% / Testing acc (1000 testing data) = 0.982
### 3. 三個三位數相加
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 99.42% / Testing acc (1000 testing data) = 0.964